In [1]:
import os
os.chdir('../')

In [2]:
os.getcwd()

'/home/nazirnayal/Documents/courses/CV_AV/HW1/cvad_hw1'

In [3]:
%load_ext autoreload
%autoreload 2

In [55]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from models.cilrs import CILRS

In [56]:
model_ci = CILRS(cond_module='command_input')

In [57]:
B = 5
x = torch.randn((B, 3, 88, 200))
v = torch.randn((B,))
c = torch.randint(0, 4, size=(B,))

In [58]:
x.shape, v.shape, c.shape

(torch.Size([5, 3, 88, 200]), torch.Size([5]), torch.Size([5]))

In [59]:
th, st, br, v_o = model_ci(x, v, c)

In [20]:
th.shape, st.shape, br.shape, v_o.shape

(torch.Size([5]), torch.Size([5]), torch.Size([5]), torch.Size([5, 1]))

In [21]:
model_br = CILRS(cond_module='branched')

In [23]:
th, st, br, v_o = model_br(x, v, c)

In [24]:
th.shape, st.shape, br.shape, v_o.shape

(torch.Size([5]), torch.Size([5]), torch.Size([5]), torch.Size([5, 1]))

In [27]:
from models.cilrs import one_hot_vector

In [30]:
c

tensor([1, 0, 2, 0, 1])

In [31]:
one_hot_vector(c, 4)

tensor([[0., 1., 0., 0.],
        [1., 0., 0., 0.],
        [0., 0., 1., 0.],
        [1., 0., 0., 0.],
        [0., 1., 0., 0.]])

# Testing Util Function

In [39]:
import numpy as np
from numpy.lib.stride_tricks import as_strided

In [43]:
def tile_array(a, b0, b1):
    r, c = a.shape                                    # number of rows/columns
    rs, cs = a.strides                                # row/column strides
    x = as_strided(a, (r, b0, c, b1), (rs, 0, cs, 0)) # view a as larger 4D array
    return x.reshape(r*b0, c*b1)                      # create new 2D array

def get_bool_vec(d, n_h):
    # first one hot encode to three possible classes
    t = np.zeros((len(d), 3))
    t[np.arange(len(d)), d+1] = 1
    # upscale to correct hidden_sz
    print(t)
    bool_vec = tile_array(t, 1, n_h//3)
    return torch.Tensor(bool_vec)

In [45]:
x = torch.randint(0, 2, size=(5,))

In [46]:
one_hot = get_bool_vec(x, 12)

[[0. 0. 1.]
 [0. 0. 1.]
 [0. 0. 1.]
 [0. 1. 0.]
 [0. 1. 0.]]


In [47]:
one_hot, one_hot.shape

(tensor([[0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 1., 1.],
         [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.],
         [0., 0., 0., 0., 1., 1., 1., 1., 0., 0., 0., 0.]]),
 torch.Size([5, 12]))

# Testing Affordance Model

In [48]:
from models.affordance_predictor import AffordancePredictor

In [49]:
model_aff = AffordancePredictor()

In [52]:
x = torch.randn((5, 3, 224, 224))
c = torch.randint(0, 4, size=(5,))

In [53]:
lane_dist, route_angle, tl_dist, tl_state = model_aff(x, c)

In [54]:
lane_dist.shape, route_angle.shape, tl_dist.shape, tl_state.shape

(torch.Size([5, 1]),
 torch.Size([5, 1]),
 torch.Size([5, 1]),
 torch.Size([5, 2]))

In [75]:
rewards = np.array([0, 10, -20, 10, 100, 10, 30, 50, 10, -100, -50, 10, 0, 0, 0, 0])
Q_2 = np.zeros(len(rewards))
Q_3 = np.zeros(len(rewards))
lr = 0.5;
num_states = len(rewards) - 4

In [76]:
for _ in range(20):
    for i in range(num_states):

        Q_2[i] = (1 - lr) * Q_2[i] + lr * (rewards[i + 2] + max(Q_2[i + 2], Q_3[i + 2]))

        Q_3[i] = (1 - lr) * Q_3[i] + lr * (rewards[i + 2] + max(Q_2[i + 3], Q_3[i + 2]))

    
    out_2 = fr'$Q_{_ + 1}(s_t, 2-step)$ '
    out_3 = fr'$Q_{_ + 1}(s_t, 3-step)$ '
    for i  in range(num_states):
        out_2 += '& '
        out_3 += '& '
        
        out_2 += str(Q_2[i] + (100 if i == 0 else 0)) + ' '
        out_3 += str(Q_3[i] + (100 if i == 0 else 0)) + ' '
        
    print(out_2 + '\\\\')
    print('\hline')
    print(out_3 + '\\\\') 
    print('\hline')
    print('------------------------------------------')
    

$Q_1(s_t, 2-step)$ & 90.0 & 5.0 & 50.0 & 5.0 & 15.0 & 25.0 & 5.0 & -50.0 & -25.0 & 5.0 & 0.0 & 0.0 \\
\hline
$Q_1(s_t, 3-step)$ & 90.0 & 5.0 & 50.0 & 5.0 & 15.0 & 25.0 & 5.0 & -50.0 & -25.0 & 5.0 & 0.0 & 0.0 \\
\hline
------------------------------------------
$Q_2(s_t, 2-step)$ & 110.0 & 10.0 & 82.5 & 20.0 & 25.0 & 12.5 & -5.0 & -72.5 & -37.5 & 7.5 & 0.0 & 0.0 \\
\hline
$Q_2(s_t, 3-step)$ & 110.0 & 15.0 & 87.5 & 20.0 & 25.0 & 25.0 & 10.0 & -72.5 & -37.5 & 7.5 & 0.0 & 0.0 \\
\hline
------------------------------------------
$Q_3(s_t, 2-step)$ & 138.75 & 20.0 & 103.75 & 27.5 & 32.5 & -5.0 & -16.25 & -82.5 & -43.75 & 8.75 & 0.0 & 0.0 \\
\hline
$Q_3(s_t, 3-step)$ & 138.75 & 25.0 & 106.25 & 27.5 & 32.5 & 18.75 & 13.75 & -82.5 & -43.75 & 8.75 & 0.0 & 0.0 \\
\hline
------------------------------------------
$Q_4(s_t, 2-step)$ & 162.5 & 28.75 & 118.125 & 28.125 & 38.125 & -18.75 & -25.0 & -86.875 & -46.875 & 9.375 & 0.0 & 0.0 \\
\hline
$Q_4(s_t, 3-step)$ & 162.5 & 33.75 & 119.375 & 28.125 & 3

In [93]:
np.set_printoptions(precision=2, suppress=False)

In [85]:
np.set_printoptions

<function numpy.set_printoptions(precision=None, threshold=None, edgeitems=None, linewidth=None, suppress=None, nanstr=None, infstr=None, formatter=None, sign=None, floatmode=None, *, legacy=None)>

In [98]:
rewards = np.array([0, 10, -20, 10, 100, 10, 30, 50, 10, -100, -50, 10, 0, 0, 0, 0])
Q_2 = np.zeros(len(rewards))
Q_3 = np.zeros(len(rewards))
lr = 0.9;
num_states = len(rewards) - 4

for _ in range(20):
    for i in range(num_states):

        Q_2[i] = (1 - lr) * Q_2[i] + lr * (rewards[i + 2] + max(Q_2[i + 2], Q_3[i + 2]))

        Q_3[i] = (1 - lr) * Q_3[i] + lr * (rewards[i + 3] + max(Q_2[i + 3], Q_3[i + 3]))

    
    out_2 = fr'$Q_{_ + 1}(s_t, 2-step)$ '
    out_3 = fr'$Q_{_ + 1}(s_t, 3-step)$ '
    for i  in range(num_states):
        out_2 += '& '
        out_3 += '& '
        
        out_2 += str(np.round(Q_2[i] + (100 if i == 0 else 0), 1)) + ' '
        out_3 += str(np.round(Q_3[i] + (100 if i == 0 else 0), 1)) + ' '
        
    print(out_2 + '\\\\')
    print('\hline')
    print(out_3 + '\\\\') 
    print('\hline')
    

$Q_1(s_t, 2-step)$ & 82.0 & 9.0 & 90.0 & 9.0 & 27.0 & 45.0 & 9.0 & -90.0 & -45.0 & 9.0 & 0.0 & 0.0 \\
\hline
$Q_1(s_t, 3-step)$ & 109.0 & 90.0 & 9.0 & 27.0 & 45.0 & 9.0 & -90.0 & -45.0 & 9.0 & 0.0 & 0.0 & 0.0 \\
\hline
$Q_2(s_t, 2-step)$ & 161.2 & 34.2 & 139.5 & 50.4 & 37.8 & 9.0 & 18.0 & -90.9 & -49.5 & 9.9 & 0.0 & 0.0 \\
\hline
$Q_2(s_t, 3-step)$ & 134.2 & 139.5 & 50.4 & 37.8 & 9.0 & 18.0 & -90.9 & -49.5 & 9.9 & 0.0 & 0.0 & 0.0 \\
\hline
$Q_3(s_t, 2-step)$ & 213.7 & 57.8 & 138.0 & 30.2 & 47.0 & 1.3 & 19.7 & -90.2 & -50.0 & 10.0 & 0.0 & 0.0 \\
\hline
$Q_3(s_t, 3-step)$ & 157.8 & 138.0 & 30.2 & 47.0 & 1.3 & 19.7 & -90.2 & -50.0 & 10.0 & 0.0 & 0.0 & 0.0 \\
\hline
$Q_4(s_t, 2-step)$ & 217.5 & 57.1 & 146.1 & 29.8 & 49.4 & 0.2 & 20.0 & -90.0 & -50.0 & 10.0 & 0.0 & 0.0 \\
\hline
$Q_4(s_t, 3-step)$ & 157.1 & 146.1 & 29.8 & 49.4 & 0.2 & 20.0 & -90.0 & -50.0 & 10.0 & 0.0 & 0.0 & 0.0 \\
\hline
$Q_5(s_t, 2-step)$ & 225.2 & 59.2 & 149.1 & 29.9 & 49.9 & 0.0 & 20.0 & -90.0 & -50.0 & 10.0 & 0.0 & 0.